In [1]:
import os
import csv
from dotenv import load_dotenv, find_dotenv
import psycopg2
import psycopg2.extras

from datetime import datetime

In [2]:
load_dotenv(find_dotenv())

host2 = os.environ.get("DB_HOST")
database2 = os.environ.get("DB_NAME")
user2 = os.environ.get("DB_USER")
password2 = os.environ.get("DB_PASSWORD")

In [3]:
connection = psycopg2.connect(host=host2, database=database2, user=user2, password=password2)
cursor = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)

In [4]:
ark_list = ["ARKF", "ARKG", "ARKK", "ARKQ", "ARKW", "ARKX", "IZRL", "PRNT"]
ark_tuple = tuple(ark_list)

cursor.execute("""SELECT * from stock_sql WHERE symbol in %s""", (ark_tuple,))
rows = cursor.fetchall()
rows

[[734, 'ARKW', 'ARK NEXT GENERATION INTERNET', 'BBG0077Q7LF9', 'ETP'],
 [5879, 'ARKK', 'ARK INNOVATION ETF', 'BBG007FJMV58', 'ETP'],
 [8015, 'ARKG', 'ARK GENOMIC REVOLUTION ETF', 'BBG007FJPCB0', 'ETP'],
 [12699, 'PRNT', 'THE 3D PRINTING ETF', 'BBG00D3D1DC6', 'ETP'],
 [15624, 'ARKX', 'ARK SPACE EXPLORATION & INNO', 'BBG00ZV1N0C6', 'ETP'],
 [23083, 'ARKQ', 'ARK AUTONOMOUS TECH & ROBOT', 'BBG0077Q7038', 'ETP'],
 [24026, 'IZRL', 'ARK ISRAEL INNOVATIVE TECHNO', 'BBG00JDV6F05', 'ETP'],
 [25103, 'ARKF', 'ARK FINTECH INNOVATION ETF', 'BBG00N7YFK60', 'ETP']]

In [5]:
today = datetime.today().strftime('%Y-%m-%d')
#today = '2021-08-03'

In [7]:
for etf in rows:
        print(etf['symbol'])

        with open(f"downloads/{today}/{etf['symbol']}.csv") as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                ticker = row[3]

                if ticker: 
                    shares = row[5]
                    weight = row[7]
                    date = row[0]
                    cursor.execute("""
                        SELECT * FROM stock_sql WHERE symbol = %s
                    """, (ticker,))
                    stock = cursor.fetchone()
                    if stock:
                        cursor.execute("""
                            INSERT INTO holdings_sql (etf_id, holding_id, dt, shares, weight)
                            VALUES (%s, %s, %s, %s, %s)
                        """, (etf['id'], stock['id'], date, shares, weight))

ARKW
ARKK
ARKG
PRNT
ARKX
ARKQ
IZRL
ARKF


In [8]:
connection.commit()